In [4]:
import collections
import glob
import json
import os
import tqdm
import openreview
import random
random.seed(34)

def get_aux_info(review_id, guest_client):
    rev_note, = guest_client.get_notes(id=review_id)
    aux_info = {}
    for key in "rating confidence experience_assessment".split():
        if key in rev_note.content:
            aux_info[key] = rev_note.content[key]
    return aux_info
            
guest_client = openreview.Client(baseurl='https://api.openreview.net')

    
annotators_worst_to_best = [
    'anno6', 'anno5', 'anno7', 'anno1', 'anno16', 'anno12', 'anno11',
    'anno2', 'anno8', 'anno13', 'anno9', 'anno10', 'anno3', 'anno0',
    'anno15', 'anno14', 'anno4']

def select_best_annotator(annotators):
    for anno in reversed(annotators_worst_to_best):
        if anno in annotators:
            return anno
def dump_obj_to_file(split, mini_name, obj):
    print("0517_split_2/"+split+"/"+mini_name)
    with open("0517_split_2/"+split+"/"+mini_name, 'w') as g:
        json.dump(obj, g)
        
os.makedirs("0517_split_2")
os.makedirs("0517_split_2/train")
os.makedirs("0517_split_2/dev")
os.makedirs("0517_split_2/test")


input_dir = "/Users/nnayak/Downloads/05-16-provisional-emnlp-release-unsplit"

forum_to_conf_map = {}
forum_to_reviews_map = collections.defaultdict(list)


for filename in sorted(glob.glob(input_dir + "/*")):
    mini_name = filename.split("/")[-1]
    with open(filename, 'r') as f:
        obj = json.load(f)
        try:
            assert len(obj["review"]) == len(obj["reviewlabels"]) and len(obj["rebuttal"]) == len(obj["rebuttallabels"])
        except AssertionError:
            print("Skipping", obj["metadata"]["review"])
            continue
        forum = obj["metadata"]["forum_id"]
        conference = obj["metadata"]["conference"]
        if forum in forum_to_conf_map:
            assert conference == forum_to_conf_map[forum]
        else:
            forum_to_conf_map[forum] = conference
        forum_to_reviews_map[forum].append(mini_name)

forum_split_map = {}
for forum, reviews in forum_to_reviews_map.items():
    annotation_counter = collections.Counter(review.split(".")[0] for review in reviews)
    
    if set(annotation_counter.values()) == set([1]):
        split = "traindev"
    else:
        split = "test"
    forum_split_map[forum] = split
    
    
selected_test_file_map = {}
for forum, split in forum_split_map.items():
    if split == "traindev":
        continue
    else:
        relevant_reviews = forum_to_reviews_map[forum]
        annotator_map = collections.defaultdict(list)
        for review_info in relevant_reviews:
            review_id, annotator, _ = review_info.split(".")
            annotator_map[review_id].append(annotator)
        for review, annotators in annotator_map.items():
            if len(annotators) == 1:
                selected_annotator = annotators[0]
            elif 'anno0' in annotators:
                selected_annotator = "anno0"
            else:
                selected_annotator = select_best_annotator(annotators)
            selected_test_file_map[review] = ".".join([review, selected_annotator, "json"])

            
possible_split = "train train train dev".split()
final_split_map = {}
for forum, current_split in forum_split_map.items():
    if current_split == "test":
        final_split_map[forum] = "test"
    else:
        final_split_map[forum] = random.choice(possible_split)
    
final_examples = []

for filename in tqdm.tqdm(glob.glob(input_dir + "/*")):
    mini_name = filename.split("/")[-1]
    with open(filename, 'r') as f:
        obj = json.load(f)
        try:
            assert len(obj["review"]) == len(obj["reviewlabels"]) and len(obj["rebuttal"]) == len(obj["rebuttallabels"])
        except AssertionError:
            print("Skipping", obj["metadata"]["review"])
            continue
        forum = obj["metadata"]["forum_id"]
        review = obj["metadata"]["review"]
        obj["metadata"].update(get_aux_info(review, guest_client))
        if forum not in final_split_map:
            print("Skipping rewrite:", review)
            continue
        split = final_split_map[forum]
        if split in 'train dev'.split():
            dump_obj_to_file(split, mini_name, obj)
            final_examples.append((split, mini_name))
        else:
            assert split == 'test'
            if review in selected_test_file_map and mini_name == selected_test_file_map[review]:
                dump_obj_to_file(split, mini_name, obj)
                final_examples.append((split, mini_name))
            else:
                print("Skipping rewrite:", review)
                
                
with open("0517_split_2/example_list.txt", 'w') as f:
    for split, mini_name in final_examples:
        review, anno, _ = mini_name.split(".")
        f.write("\t".join([review, anno, split]) + "\n")

Skipping H1xX2I782m
Skipping HkeQCAq6hQ
Skipping Hygsieas2X
Skipping Hyx3bbVc27
Skipping S1l9L6uQqS
Skipping Skg4J4xt27


  0%|          | 0/647 [00:00<?, ?it/s]

Skipping SygP4mws3Q
Skipping r1gjMgfaYH
Skipping rJgHSAbAFH
Skipping rJlsvSSchm
Skipping rkgfiCmT5r


  0%|          | 1/647 [00:00<02:58,  3.62it/s]

0517_split_2/test/SyeMmx63FS.anno0.json
Skipping HkeQCAq6hQ


...

0517_split_2/test/rJgHSAbAFH.anno7.json


100%|█████████▉| 646/647 [02:57<00:00,  3.69it/s]

0517_split_2/test/rylen8dl5H.anno0.json


100%|██████████| 647/647 [02:57<00:00,  3.64it/s]

Skipping rewrite: Bke6usYzKr
